# 🚀 Day 2: Shared Memory - Your On-Chip Scratchpad

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-02/day-2-shared-memory.ipynb)

## Learning Philosophy

> **CUDA C++ First, Python/Numba as Optional Backup**

This notebook shows:
1. **CUDA C++ code** - The PRIMARY implementation you should learn
2. **Python/Numba code** - OPTIONAL for quick interactive testing in Colab

---

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
# Python/Numba is OPTIONAL - for quick interactive testing only
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

print("\n⚠️  Remember: CUDA C++ code is the PRIMARY learning material!")

# Day 2: Shared Memory - Your On-Chip Scratchpad

Yesterday you learned that non-coalesced access kills performance. Today's solution: **shared memory**!

## Learning Objectives
- Understand shared memory architecture
- Declare and use shared memory (`__shared__` in CUDA C++)
- Master thread synchronization (`__syncthreads()`)
- Implement tile-based algorithms
- Fix the matrix transpose problem

---

## 1. What is Shared Memory?

```
GPU Memory Hierarchy:
┌────────────────────────────────────────────────┐
│                Streaming Multiprocessor (SM)   │
│  ┌──────────────────────────────────────────┐  │
│  │    REGISTERS (per thread, ~255 each)     │  │  ← Fastest
│  └──────────────────────────────────────────┘  │
│  ┌──────────────────────────────────────────┐  │
│  │    SHARED MEMORY (per block, 48-164 KB)  │  │  ← Very fast (~5 cycles)
│  │    ┌─────────────────────────────────┐   │  │
│  │    │ Block 0 threads can share data  │   │  │
│  │    │ Thread 0 ←→ Thread 1 ←→ Thread N│   │  │
│  │    └─────────────────────────────────┘   │  │
│  └──────────────────────────────────────────┘  │
└────────────────────────────────────────────────┘
                      ↕ (~400 cycles latency)
┌────────────────────────────────────────────────┐
│           GLOBAL MEMORY (4-80 GB)              │  ← Slow but large
└────────────────────────────────────────────────┘
```

### Key Properties:
| Property | Global Memory | Shared Memory |
|----------|---------------|---------------|
| Size | 4-80 GB | 48-164 KB per SM |
| Latency | ~400 cycles | ~5 cycles |
| Bandwidth | 200-900 GB/s | ~10 TB/s |
| Scope | All threads | Threads in same block |
| Lifetime | Until freed | Until block completes |

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile shared_mem_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Shared memory in CUDA C++
__global__ void sharedMemoryDemo(float* output, int n) {
    // Static allocation (size known at compile time)
    __shared__ float tile[256];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Each thread writes to shared memory
    tile[tid] = (float)gid;
    
    // REQUIRED: Wait for all threads to finish writing
    __syncthreads();
    
    // Now safe to read other threads' data
    // Read from reversed position
    int reversed_tid = blockDim.x - 1 - tid;
    if (gid < n) {
        output[gid] = tile[reversed_tid];
    }
}

// Dynamic shared memory example
__global__ void dynamicSharedDemo(float* input, float* output, int n) {
    // Dynamic allocation (size passed at kernel launch)
    extern __shared__ float dynamicTile[];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load data to shared memory
    if (gid < n) {
        dynamicTile[tid] = input[gid];
    }
    __syncthreads();
    
    // Compute using shared memory
    if (gid < n && tid > 0 && tid < blockDim.x - 1) {
        // 3-point stencil using shared memory
        output[gid] = (dynamicTile[tid-1] + dynamicTile[tid] + dynamicTile[tid+1]) / 3.0f;
    }
}

int main() {
    const int N = 256;
    const int bytes = N * sizeof(float);
    
    // Allocate memory
    float *h_input = (float*)malloc(bytes);
    float *h_output = (float*)malloc(bytes);
    float *d_input, *d_output;
    cudaMalloc(&d_input, bytes);
    cudaMalloc(&d_output, bytes);
    
    // Initialize
    for (int i = 0; i < N; i++) {
        h_input[i] = (float)i;
    }
    cudaMemcpy(d_input, h_input, bytes, cudaMemcpyHostToDevice);
    
    printf("=== Shared Memory Demonstration ===\n\n");
    
    // Test static shared memory (reverse within block)
    sharedMemoryDemo<<<1, 256>>>(d_output, N);
    cudaMemcpy(h_output, d_output, bytes, cudaMemcpyDeviceToHost);
    
    printf("Static shared memory (block reverse):\n");
    printf("  Input:  [0, 1, 2, 3, ...]\n");
    printf("  Output: [%.0f, %.0f, %.0f, %.0f, ...]\n\n", 
           h_output[0], h_output[1], h_output[2], h_output[3]);
    
    // Test dynamic shared memory (3-point stencil)
    // Launch with dynamic shared memory size
    size_t sharedMemSize = N * sizeof(float);
    dynamicSharedDemo<<<1, 256, sharedMemSize>>>(d_input, d_output, N);
    cudaMemcpy(h_output, d_output, bytes, cudaMemcpyDeviceToHost);
    
    printf("Dynamic shared memory (3-point stencil):\n");
    printf("  Input[10]:  %.1f\n", h_input[10]);
    printf("  Output[10]: %.1f (average of %.1f, %.1f, %.1f)\n",
           h_output[10], h_input[9], h_input[10], h_input[11]);
    
    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    
    printf("\n✅ Shared memory demonstration complete!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o shared_mem_demo shared_mem_demo.cu
!./shared_mem_demo

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
import numpy as np
from numba import cuda
import math
import time

device = cuda.get_current_device()
print(f"GPU: {device.name.decode()}")
print(f"Shared memory per block: {device.MAX_SHARED_MEMORY_PER_BLOCK / 1024:.1f} KB")
print(f"Max threads per block: {device.MAX_THREADS_PER_BLOCK}")

## 2. Declaring Shared Memory in Numba

In Numba CUDA, we use `cuda.shared.array()` to allocate shared memory:

```python
@cuda.jit
def my_kernel(...):
    # Allocate shared memory - size must be constant!
    shared_data = cuda.shared.array(shape=256, dtype=numba.float32)
    
    # All threads in the block can access shared_data
    tid = cuda.threadIdx.x
    shared_data[tid] = ...  # Write
    
    cuda.syncthreads()  # Synchronize before reading
    
    val = shared_data[other_tid]  # Read another thread's data
```

In [ ]:
from numba import float32, int32

# Basic shared memory example: Reverse an array within each block
@cuda.jit
def reverse_block(input_arr, output_arr):
    """
    Each block reverses its portion of the array using shared memory.
    """
    # Allocate shared memory for the block
    shared = cuda.shared.array(shape=256, dtype=float32)
    
    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    block_size = cuda.blockDim.x
    
    # Load data from global memory into shared memory
    if gid < input_arr.size:
        shared[tid] = input_arr[gid]
    
    # CRITICAL: Wait for all threads to finish loading
    cuda.syncthreads()
    
    # Read from reversed position in shared memory
    if gid < input_arr.size:
        reversed_tid = block_size - 1 - tid
        output_arr[gid] = shared[reversed_tid]

# Test it
n = 256
input_arr = np.arange(n, dtype=np.float32)
output_arr = np.zeros(n, dtype=np.float32)

input_d = cuda.to_device(input_arr)
output_d = cuda.to_device(output_arr)

reverse_block[1, 256](input_d, output_d)
result = output_d.copy_to_host()

print("Input:  ", input_arr[:8], "...")
print("Output: ", result[:8], "...")
print("\n✅ Reversed correctly:", np.array_equal(result, input_arr[::-1]))

## 3. The Importance of `cuda.syncthreads()`

Threads in a block execute **concurrently but not simultaneously**. Without synchronization:

```
WITHOUT syncthreads():          WITH syncthreads():
                                
Thread 0: write shared[0]       Thread 0: write shared[0]
Thread 1: read shared[0] ❌     ───── syncthreads() ─────
          (might get old value)  Thread 1: read shared[0] ✅
```

### Rules:
1. **All threads in the block must reach the same `syncthreads()`**
2. Call `syncthreads()` after writes, before reads from other threads
3. Never put `syncthreads()` inside a conditional that not all threads take!

In [ ]:
# Demonstrating the need for syncthreads

@cuda.jit
def sum_neighbors_wrong(arr, out):
    """WRONG: No synchronization - race condition!"""
    shared = cuda.shared.array(shape=256, dtype=float32)
    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    
    if gid < arr.size:
        shared[tid] = arr[gid]
    
    # BUG: No syncthreads! Neighbors might not have loaded yet!
    
    if gid < arr.size and tid > 0 and tid < 255:
        out[gid] = shared[tid-1] + shared[tid] + shared[tid+1]

@cuda.jit
def sum_neighbors_correct(arr, out):
    """CORRECT: Proper synchronization"""
    shared = cuda.shared.array(shape=256, dtype=float32)
    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    
    if gid < arr.size:
        shared[tid] = arr[gid]
    
    # Wait for all threads to load their values
    cuda.syncthreads()
    
    if gid < arr.size and tid > 0 and tid < 255:
        out[gid] = shared[tid-1] + shared[tid] + shared[tid+1]

# Test both
n = 256
arr = np.ones(n, dtype=np.float32)

out_wrong = cuda.device_array(n, dtype=np.float32)
out_correct = cuda.device_array(n, dtype=np.float32)

arr_d = cuda.to_device(arr)

sum_neighbors_correct[1, 256](arr_d, out_correct)
result = out_correct.copy_to_host()

print("Sum of neighbors (should be 3.0 for middle elements):")
print(f"  Element 10: {result[10]}")
print(f"  Element 100: {result[100]}")
print(f"  Element 200: {result[200]}")

## 4. Tile-Based Algorithms

The most common shared memory pattern: **load a tile, process it, store results**.

```
TILING PATTERN:

Global Memory (slow):       Shared Memory (fast):
┌─────────────────────┐     ┌──────────────┐
│████████░░░░░░░░░░░░│ ──→ │████████████│ Tile 0
│░░░░░░░░████████░░░░│     └──────────────┘
│░░░░░░░░░░░░░░░░████│         ↓ process
└─────────────────────┘         ↓ in shared
                               ↓ memory
1. Load tile from global      
2. syncthreads()              
3. Process using shared mem   
4. syncthreads()              
5. Store results to global    
```

### 🔷 CUDA C++ Tiled Stencil (Primary)

In [ ]:
%%writefile stencil_tiled.cu
#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_SIZE 256
#define RADIUS 1

// Naive: 3 global memory reads per thread
__global__ void stencilNaive(const float* input, float* output, int n) {
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (gid > 0 && gid < n - 1) {
        output[gid] = (input[gid-1] + input[gid] + input[gid+1]) / 3.0f;
    }
}

// Tiled: Load to shared memory, reuse for stencil computation
__global__ void stencilTiled(const float* input, float* output, int n) {
    __shared__ float tile[TILE_SIZE + 2 * RADIUS];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load main element
    if (gid < n) {
        tile[tid + RADIUS] = input[gid];
    }
    
    // Load halo elements
    if (tid == 0 && gid > 0) {
        tile[0] = input[gid - 1];
    }
    if (tid == blockDim.x - 1 && gid < n - 1) {
        tile[TILE_SIZE + RADIUS] = input[gid + 1];
    }
    
    __syncthreads();
    
    // Compute stencil from shared memory
    if (gid > 0 && gid < n - 1) {
        output[gid] = (tile[tid] + tile[tid + 1] + tile[tid + 2]) / 3.0f;
    }
}

int main() {
    const int N = 10000000;
    const size_t bytes = N * sizeof(float);
    
    float *h_input = (float*)malloc(bytes);
    float *h_output = (float*)malloc(bytes);
    
    for (int i = 0; i < N; i++) h_input[i] = (float)i;
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, bytes);
    cudaMalloc(&d_output, bytes);
    cudaMemcpy(d_input, h_input, bytes, cudaMemcpyHostToDevice);
    
    int threads = TILE_SIZE;
    int blocks = (N + threads - 1) / threads;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    printf("=== Stencil: Naive vs Tiled ===\n");
    printf("Array size: %d elements\n\n", N);
    
    // Benchmark naive
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        stencilNaive<<<blocks, threads>>>(d_input, d_output, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float naive_ms;
    cudaEventElapsedTime(&naive_ms, start, stop);
    
    // Benchmark tiled
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        stencilTiled<<<blocks, threads>>>(d_input, d_output, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float tiled_ms;
    cudaEventElapsedTime(&tiled_ms, start, stop);
    
    printf("Naive (global memory): %.3f ms\n", naive_ms / 100);
    printf("Tiled (shared memory): %.3f ms\n", tiled_ms / 100);
    printf("Speedup: %.2fx\n", naive_ms / tiled_ms);
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    
    return 0;
}

In [ ]:
!nvcc -o stencil_tiled stencil_tiled.cu && ./stencil_tiled

In [ ]:
# 1D Stencil with tiling (3-point stencil: left + center + right)

TILE_SIZE = 256
STENCIL_RADIUS = 1

@cuda.jit
def stencil_1d_naive(input_arr, output_arr, n):
    """Naive stencil: 3 global memory reads per thread"""
    gid = cuda.grid(1)
    
    if gid > 0 and gid < n - 1:
        # 3 global memory accesses (slow!)
        output_arr[gid] = (input_arr[gid-1] + 
                          input_arr[gid] + 
                          input_arr[gid+1]) / 3.0

@cuda.jit
def stencil_1d_tiled(input_arr, output_arr, n):
    """Tiled stencil: Load once, reuse from shared memory"""
    # Shared memory includes "halo" cells for stencil
    shared = cuda.shared.array(shape=TILE_SIZE + 2 * STENCIL_RADIUS, dtype=float32)
    
    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    
    # Load main tile element
    if gid < n:
        shared[tid + STENCIL_RADIUS] = input_arr[gid]
    
    # Load halo (left and right boundary elements)
    if tid == 0 and gid > 0:
        shared[0] = input_arr[gid - 1]  # Left halo
    if tid == cuda.blockDim.x - 1 and gid < n - 1:
        shared[tid + STENCIL_RADIUS + 1] = input_arr[gid + 1]  # Right halo
    
    cuda.syncthreads()
    
    # Compute stencil from shared memory (fast!)
    if gid > 0 and gid < n - 1:
        output_arr[gid] = (shared[tid] + 
                          shared[tid + 1] + 
                          shared[tid + 2]) / 3.0

def benchmark_stencil(n, iterations=100):
    """Compare naive vs tiled stencil"""
    input_arr = cuda.to_device(np.random.randn(n).astype(np.float32))
    output_naive = cuda.device_array(n, dtype=np.float32)
    output_tiled = cuda.device_array(n, dtype=np.float32)
    
    threads = TILE_SIZE
    blocks = math.ceil(n / threads)
    
    # Warmup
    stencil_1d_naive[blocks, threads](input_arr, output_naive, n)
    stencil_1d_tiled[blocks, threads](input_arr, output_tiled, n)
    cuda.synchronize()
    
    # Benchmark naive
    start = time.perf_counter()
    for _ in range(iterations):
        stencil_1d_naive[blocks, threads](input_arr, output_naive, n)
    cuda.synchronize()
    naive_time = (time.perf_counter() - start) / iterations
    
    # Benchmark tiled
    start = time.perf_counter()
    for _ in range(iterations):
        stencil_1d_tiled[blocks, threads](input_arr, output_tiled, n)
    cuda.synchronize()
    tiled_time = (time.perf_counter() - start) / iterations
    
    # Verify correctness
    naive_result = output_naive.copy_to_host()
    tiled_result = output_tiled.copy_to_host()
    correct = np.allclose(naive_result[1:-1], tiled_result[1:-1])
    
    return naive_time, tiled_time, correct

n = 10_000_000
naive_time, tiled_time, correct = benchmark_stencil(n)

print(f"1D Stencil Benchmark (N = {n:,})")
print("=" * 45)
print(f"Naive (global mem):  {naive_time*1000:.3f} ms")
print(f"Tiled (shared mem):  {tiled_time*1000:.3f} ms")
print(f"Speedup:             {naive_time/tiled_time:.2f}x")
print(f"Results match:       {correct}")

## 5. Fixing Matrix Transpose with Shared Memory

Yesterday's problem: You can't have both coalesced reads AND coalesced writes.

**Solution:** Use shared memory as an intermediate buffer!

```
1. Coalesced read from input → shared memory (tile)
2. Transpose the tile in shared memory
3. Coalesced write from shared memory → output
```

### 🔷 CUDA C++ Matrix Transpose with Shared Memory (Primary)

In [ ]:
%%writefile transpose_shared.cu
#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_DIM 32

// Naive: coalesced reads, non-coalesced writes
__global__ void transposeNaive(const float* input, float* output, int rows, int cols) {
    int col = blockIdx.x * TILE_DIM + threadIdx.x;
    int row = blockIdx.y * TILE_DIM + threadIdx.y;
    
    if (row < rows && col < cols) {
        output[col * rows + row] = input[row * cols + col];
    }
}

// Shared memory: BOTH reads and writes are coalesced!
__global__ void transposeShared(const float* input, float* output, int rows, int cols) {
    __shared__ float tile[TILE_DIM][TILE_DIM];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    // Global indices for reading
    int row = blockIdx.y * TILE_DIM + ty;
    int col = blockIdx.x * TILE_DIM + tx;
    
    // Step 1: Coalesced read from global to shared
    if (row < rows && col < cols) {
        tile[ty][tx] = input[row * cols + col];
    }
    
    __syncthreads();
    
    // Step 2: Compute transposed output position
    int out_row = blockIdx.x * TILE_DIM + ty;
    int out_col = blockIdx.y * TILE_DIM + tx;
    
    // Step 3: Coalesced write from shared to global
    if (out_row < cols && out_col < rows) {
        output[out_row * rows + out_col] = tile[tx][ty];
    }
}

int main() {
    const int ROWS = 4096;
    const int COLS = 4096;
    const size_t bytes = ROWS * COLS * sizeof(float);
    
    float *h_input = (float*)malloc(bytes);
    float *h_output = (float*)malloc(bytes);
    for (int i = 0; i < ROWS * COLS; i++) h_input[i] = (float)i;
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, bytes);
    cudaMalloc(&d_output, bytes);
    cudaMemcpy(d_input, h_input, bytes, cudaMemcpyHostToDevice);
    
    dim3 threads(TILE_DIM, TILE_DIM);
    dim3 blocks((COLS + TILE_DIM - 1) / TILE_DIM, (ROWS + TILE_DIM - 1) / TILE_DIM);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    printf("=== Matrix Transpose: Naive vs Shared Memory ===\n");
    printf("Matrix: %d x %d\n\n", ROWS, COLS);
    
    // Benchmark naive
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        transposeNaive<<<blocks, threads>>>(d_input, d_output, ROWS, COLS);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float naive_ms;
    cudaEventElapsedTime(&naive_ms, start, stop);
    float naive_bw = (2 * bytes * 100) / (naive_ms / 1000) / 1e9;
    
    // Benchmark shared
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        transposeShared<<<blocks, threads>>>(d_input, d_output, ROWS, COLS);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float shared_ms;
    cudaEventElapsedTime(&shared_ms, start, stop);
    float shared_bw = (2 * bytes * 100) / (shared_ms / 1000) / 1e9;
    
    printf("Naive:  %.2f ms (%.1f GB/s)\n", naive_ms / 100, naive_bw);
    printf("Shared: %.2f ms (%.1f GB/s)\n", shared_ms / 100, shared_bw);
    printf("Speedup: %.2fx\n", naive_ms / shared_ms);
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    
    return 0;
}

In [ ]:
!nvcc -o transpose_shared transpose_shared.cu && ./transpose_shared

In [ ]:
TILE_DIM = 32  # Must match block size for this simple version

@cuda.jit
def transpose_naive(input_mat, output_mat):
    """Naive transpose: coalesced reads, non-coalesced writes"""
    col, row = cuda.grid(2)
    rows, cols = input_mat.shape
    
    if row < rows and col < cols:
        output_mat[col, row] = input_mat[row, col]

@cuda.jit
def transpose_shared(input_mat, output_mat):
    """
    Shared memory transpose: BOTH reads and writes are coalesced!
    """
    # Shared memory tile
    tile = cuda.shared.array(shape=(TILE_DIM, TILE_DIM), dtype=float32)
    
    # Thread indices within block
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # Global indices for reading (coalesced: threads read along row)
    row = cuda.blockIdx.y * TILE_DIM + ty
    col = cuda.blockIdx.x * TILE_DIM + tx
    
    rows, cols = input_mat.shape
    
    # Step 1: Coalesced read from global to shared
    if row < rows and col < cols:
        tile[ty, tx] = input_mat[row, col]
    
    cuda.syncthreads()
    
    # Step 2: Compute transposed output position
    # Block indices are swapped for output
    out_row = cuda.blockIdx.x * TILE_DIM + ty  # Note: blockIdx.x, not y!
    out_col = cuda.blockIdx.y * TILE_DIM + tx  # Note: blockIdx.y, not x!
    
    # Step 3: Coalesced write from shared to global
    # Note: we read from tile[tx, ty] (transposed indices!)
    if out_row < cols and out_col < rows:
        output_mat[out_row, out_col] = tile[tx, ty]

def benchmark_transpose(size, iterations=50):
    """Compare naive vs shared memory transpose"""
    input_mat = cuda.to_device(np.random.randn(size, size).astype(np.float32))
    output_naive = cuda.device_array((size, size), dtype=np.float32)
    output_shared = cuda.device_array((size, size), dtype=np.float32)
    
    threads = (TILE_DIM, TILE_DIM)
    blocks = (math.ceil(size / TILE_DIM), math.ceil(size / TILE_DIM))
    
    # Warmup
    transpose_naive[blocks, threads](input_mat, output_naive)
    transpose_shared[blocks, threads](input_mat, output_shared)
    cuda.synchronize()
    
    # Benchmark naive
    start = time.perf_counter()
    for _ in range(iterations):
        transpose_naive[blocks, threads](input_mat, output_naive)
    cuda.synchronize()
    naive_time = (time.perf_counter() - start) / iterations
    
    # Benchmark shared
    start = time.perf_counter()
    for _ in range(iterations):
        transpose_shared[blocks, threads](input_mat, output_shared)
    cuda.synchronize()
    shared_time = (time.perf_counter() - start) / iterations
    
    # Verify
    naive_result = output_naive.copy_to_host()
    shared_result = output_shared.copy_to_host()
    input_host = input_mat.copy_to_host()
    
    correct_naive = np.allclose(naive_result, input_host.T)
    correct_shared = np.allclose(shared_result, input_host.T)
    
    # Bandwidth
    bytes_moved = 2 * size * size * 4  # Read + write
    naive_bw = bytes_moved / naive_time / 1e9
    shared_bw = bytes_moved / shared_time / 1e9
    
    return naive_time, shared_time, naive_bw, shared_bw, correct_naive and correct_shared

# Run benchmark
size = 4096
naive_time, shared_time, naive_bw, shared_bw, correct = benchmark_transpose(size)

print(f"Matrix Transpose: {size} × {size}")
print("=" * 50)
print(f"Naive:          {naive_time*1000:.3f} ms  ({naive_bw:.1f} GB/s)")
print(f"Shared memory:  {shared_time*1000:.3f} ms  ({shared_bw:.1f} GB/s)")
print(f"Speedup:        {naive_time/shared_time:.2f}x")
print(f"Results correct: {correct}")

## 🎯 Exercises

### Exercise 1: Shared Memory Array Shift
Shift all elements in an array by 1 position using shared memory.

### Exercise 2: 2D Tile Sum
Sum all elements in each 16x16 tile of a 2D array.

---

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile shared_exercises.cu
// shared_exercises.cu - Shared memory exercises
#include <stdio.h>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256
#define TILE_SIZE 16

// Exercise 1: Array Shift using shared memory
// Shift all elements right by 1, with wrap-around
__global__ void arrayShift(const float* input, float* output, int n) {
    __shared__ float tile[BLOCK_SIZE + 1];  // Extra element for wrap
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load to shared memory
    if (gid < n) {
        tile[tid + 1] = input[gid];  // Shift right by storing at tid+1
    }
    
    // First thread loads the last element of previous block (or wrap)
    if (tid == 0) {
        int prevIdx = (gid == 0) ? n - 1 : gid - 1;
        tile[0] = input[prevIdx];
    }
    
    __syncthreads();
    
    // Write shifted output
    if (gid < n) {
        output[gid] = tile[tid];
    }
}

// Exercise 2: 2D Tile Sum - Sum all elements in each 16x16 tile
__global__ void tileSum(const float* input, float* output, int width, int height) {
    __shared__ float tile[TILE_SIZE][TILE_SIZE];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + tx;
    int row = blockIdx.y * TILE_SIZE + ty;
    
    // Load tile to shared memory
    if (row < height && col < width) {
        tile[ty][tx] = input[row * width + col];
    } else {
        tile[ty][tx] = 0.0f;
    }
    
    __syncthreads();
    
    // Parallel reduction within tile (using thread 0)
    if (tx == 0 && ty == 0) {
        float sum = 0.0f;
        for (int i = 0; i < TILE_SIZE; i++) {
            for (int j = 0; j < TILE_SIZE; j++) {
                sum += tile[i][j];
            }
        }
        output[blockIdx.y * gridDim.x + blockIdx.x] = sum;
    }
}

int main() {
    printf("=== Exercise 1: Array Shift ===\n");
    const int N = 16;
    float h_input[N], h_output[N];
    
    for (int i = 0; i < N; i++) h_input[i] = (float)i;
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    
    arrayShift<<<1, N>>>(d_input, d_output, N);
    cudaMemcpy(h_output, d_output, N * sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("Input:  ");
    for (int i = 0; i < N; i++) printf("%.0f ", h_input[i]);
    printf("\nOutput: ");
    for (int i = 0; i < N; i++) printf("%.0f ", h_output[i]);
    printf("\n(Each element shifted right by 1)\n\n");
    
    printf("=== Exercise 2: 2D Tile Sum ===\n");
    const int W = 32, H = 32;
    float h_matrix[W * H];
    for (int i = 0; i < W * H; i++) h_matrix[i] = 1.0f;
    
    int numTilesX = (W + TILE_SIZE - 1) / TILE_SIZE;
    int numTilesY = (H + TILE_SIZE - 1) / TILE_SIZE;
    float *h_sums = (float*)malloc(numTilesX * numTilesY * sizeof(float));
    
    float *d_matrix, *d_sums;
    cudaMalloc(&d_matrix, W * H * sizeof(float));
    cudaMalloc(&d_sums, numTilesX * numTilesY * sizeof(float));
    cudaMemcpy(d_matrix, h_matrix, W * H * sizeof(float), cudaMemcpyHostToDevice);
    
    dim3 threads(TILE_SIZE, TILE_SIZE);
    dim3 blocks(numTilesX, numTilesY);
    tileSum<<<blocks, threads>>>(d_matrix, d_sums, W, H);
    cudaMemcpy(h_sums, d_sums, numTilesX * numTilesY * sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("Tile sums (each 16x16 tile, all 1s):\n");
    for (int i = 0; i < numTilesX * numTilesY; i++) {
        printf("Tile %d: %.0f (expected 256)\n", i, h_sums[i]);
    }
    
    cudaFree(d_input); cudaFree(d_output);
    cudaFree(d_matrix); cudaFree(d_sums);
    free(h_sums);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o shared_exercises shared_exercises.cu && ./shared_exercises

### 🔶 Python/Numba Exercises (Optional)

In [ ]:
# Exercise 1: Array shift with shared memory

@cuda.jit
def shift_array(input_arr, output_arr, n, shift):
    """
    Shift array elements by 'shift' positions.
    output[i] = input[(i + shift) % n]
    
    Use shared memory to load a tile, then read from shifted positions.
    """
    shared = cuda.shared.array(shape=256, dtype=float32)
    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    
    # TODO: Implement the shift using shared memory
    # 1. Load tile into shared memory
    # 2. syncthreads()
    # 3. Read from shifted position in shared memory
    # 4. Write to output
    
    pass

# Test your implementation
# n = 256
# shift = 5
# ...

### Exercise 2: 2D Tile Sum

Compute the sum of each 16×16 tile in a matrix.

In [ ]:
# Exercise 2: 2D Tile Sum

TILE_SIZE_2D = 16

@cuda.jit
def tile_sum(matrix, tile_sums, rows, cols):
    """
    Each block computes the sum of its 16×16 tile.
    tile_sums[block_y, block_x] = sum of that tile
    """
    shared = cuda.shared.array(shape=(TILE_SIZE_2D, TILE_SIZE_2D), dtype=float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # TODO: Implement tile sum
    # 1. Load tile into shared memory
    # 2. syncthreads()
    # 3. Perform parallel reduction to sum the tile
    # 4. Thread (0,0) writes result to tile_sums
    
    pass

# Test your implementation
# ...

## 📝 Key Takeaways

### Shared Memory Essentials:

1. **Declaration:**
   ```python
   shared = cuda.shared.array(shape=SIZE, dtype=float32)
   ```

2. **Always synchronize:**
   ```python
   cuda.syncthreads()  # After writes, before reads
   ```

3. **Tiling pattern:**
   - Load tile from global → shared
   - `syncthreads()`
   - Process in shared memory
   - `syncthreads()` (if needed)
   - Store to global

4. **Use cases:**
   - Data reuse (stencils, convolutions)
   - Coalescing fix (transpose)
   - Thread cooperation (reduction)

### Performance Tips:
- Shared memory: ~100x faster than global
- But limited to 48-164 KB per block
- Tomorrow: Bank conflicts can slow it down!

---

### 📚 Next Up: Day 3 - Bank Conflicts
- How shared memory is organized in banks
- What causes bank conflicts
- Padding and access pattern tricks

---

### 🔗 Resources
- [CUDA Memory Guide](../../cuda-programming-guide/02-basics/cuda-memory.md)
- [Understanding Memory](../../cuda-programming-guide/02-basics/understanding-memory.md)